<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/NLU_12162020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install -qq transformers
!pip install twokenize

In [48]:
import twokenize
twokenize.tokenize("lol ly x0x0,:D")

['lol', 'ly', 'x0x0', ',', ':D']

In [49]:
import torch
import numpy as np
import pandas as pd

In [50]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [51]:
possible_labels=df.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [52]:
df['a_label']=df.subtask_a.replace(label_dict)
df.head()
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1


In [53]:
!pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.6.0)


In [54]:
!pip install wordsegment

In [55]:
import emoji
import re
def foremoji(text):
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text:
    w=re.sub(r":.", "<emoji>", w)
    w=re.sub(r".:", "<emoji>", w)
    w=re.sub(r"_"," ",w)
    new_text.append(w)
  text = " ".join(new_text)
  return text

In [56]:
from wordsegment import load
load()

In [57]:
from wordsegment import segment
import emoji
import re
def foremoji(text):
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text:
    w=re.sub(r":.", "<emoji>", w)
    w=re.sub(r".:", "<emoji>", w)
    w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return text

In [58]:
df['emoji_tweet']=df.tweet.apply(foremoji)

In [59]:
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,emoji_tweet
id,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,user she should ask a few native americans wha...
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,user user go home your e drunk user maga trump...
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,amazon is investigating chinese employees who ...
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,user someone shouldve taken this piece of shit...
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,user user obama wanted liberals amp illegals t...


In [60]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  df.index.values,
  df.a_label.values,
  test_size=0.2,
  random_state=RANDOM_SEED,
  stratify=df.a_label.values
)

In [61]:
df['data_type']=['not_set']*df.shape[0]

df.loc[X_train,'data_type']='train'
df.loc[X_val,'data_type']='val'
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,emoji_tweet,data_type
id,,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,user she should ask a few native americans wha...,train
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,user user go home your e drunk user maga trump...,val
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,amazon is investigating chinese employees who ...,train
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,user someone shouldve taken this piece of shit...,train
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,user user obama wanted liberals amp illegals t...,train


In [62]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [63]:

from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [64]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(28997, 768)

In [65]:



encoder_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)


In [66]:
inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].a_label.values)

In [67]:

from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [68]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [69]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [70]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [71]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [72]:
epochs=3

In [73]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.49473977310644895
Validation loss: 0.43583376997386114
F1 Score (Weighted): 0.7930516988770582



Epoch 2
Training loss: 0.38681031164806057
Validation loss: 0.4511141439967127
F1 Score (Weighted): 0.7922820430407886



Epoch 3
Training loss: 0.31725952966740845
Validation loss: 0.5079621205846947
F1 Score (Weighted): 0.7873676110757433



In [74]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [75]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_l.head()

,subtask_a
id,
15923,OFF
27014,NOT
30530,NOT
13876,NOT
60133,OFF


In [76]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [77]:
possible_labels=test.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [78]:
test['a_label']=test.subtask_a.replace(label_dict)
test.head()

,tweet,subtask_a,a_label
id,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1
13876,#Watching #Boomer getting the news that she is...,NOT,1
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0


In [79]:
test['data_type']=['test']*test.shape[0]

In [80]:
test.head()

,tweet,subtask_a,a_label,data_type
id,,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0,test
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1,test
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1,test
13876,#Watching #Boomer getting the news that she is...,NOT,1,test
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0,test


In [81]:
test['emoji_tweet']=test.tweet.apply(foremoji)

In [82]:
test.head()

,tweet,subtask_a,a_label,data_type,emoji_tweet
id,,,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0,test,who is q wheres the server dump nike dec las f...
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1,test,constitution day is revered by conservatives h...
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1,test,foxnews nra maga potus trump 2nd amendment rnc...
13876,#Watching #Boomer getting the news that she is...,NOT,1,test,watching boomer getting the news that she is s...
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0,test,no pasar ae moji unity demo to oppose the far ...


In [83]:

encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

In [84]:
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [91]:

model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [92]:
_, predictions, true_test = evaluate(dataloader_test)

In [93]:
accuracy_per_class(predictions, true_test)

Class: OFF
Accuracy: 174/240

Class: NOT
Accuracy: 556/620



In [94]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.73109   0.72500   0.72803       240
           1    0.89389   0.89677   0.89533       620

    accuracy                        0.84884       860
   macro avg    0.81249   0.81089   0.81168       860
weighted avg    0.84846   0.84884   0.84864       860

